# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült, 
az alábbi forrás alapján: 
https://github.com/bradleypallen/keras-movielens-cf

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)
</PRE>

In [1]:
import pandas as pd
import numpy as np
from CFModel import CFModel # letölthető innen: https://github.com/bradleypallen/keras-movielens-cf/blob/master/CFModel.py

Using Theano backend.


## Konstansok (előre elkészített adatállományok és a beágyazás dimenziószáma), valamint egy előtanított modell


In [2]:
RATINGS_CSV_FILE = 'ml1m_ratings.csv'
USERS_CSV_FILE = 'ml1m_users.csv'
MOVIES_CSV_FILE = 'ml1m_movies.csv'
MODEL_WEIGHTS_FILE = 'ml1m_weights.h5'
EMBEDDING_DIM = 120

## MovieLens 1M betöltése

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'movieid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
print (len(ratings), 'értékelés betöltve.')

1000209 értékelés betöltve.


In [4]:
users = pd.read_csv(USERS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'gender', 'zipcode', 'age_desc', 'occ_desc'])
print (len(users), 'felhasználó a', max_userid, 'felhasználó közül betöltve.')

6040 felhasználó a 6040 felhasználó közül betöltve.


In [5]:
movies = pd.read_csv(MOVIES_CSV_FILE, sep='\t', encoding='latin-1', usecols=['movieid', 'title', 'genre'])
print (len(movies), 'film a', max_movieid, 'film közül betöltve.')

3883 film a 3952 film közül betöltve.


## Filmajánlás egy véletlenszerűen választott felhasználónak
A felhasználói értékelést jelezzük előre a filmekre.

In [6]:
# keras.layers.embeddings.Embedding(input_dim, output_dim, init='uniform', input_length=None, W_regularizer=None, activity_regularizer=None, W_constraint=None, mask_zero=False, weights=None, dropout=0.0)
# 2 ágon torténik közös embedding: userekre és itemekre, K_FACTORS dimenzióra
trained_model = CFModel(max_userid, max_movieid, EMBEDDING_DIM)

In [7]:
# 2 súlymátrixra van szükségünk, egy user->embeddingre és egy item->embeddingre
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [8]:
# Egy véletlen felhasználó és adatai
test_user=np.random.randint(0,max_userid)
users[users['userid'] == test_user]

,userid,gender,zipcode,age_desc,occ_desc
4397,4398,F,43612,18-24,college/grad student


In [9]:
# Egy véletlen film és adatai
test_movie=np.random.randint(0,max_movieid)
movies[movies['movieid'] == test_movie]

,movieid,title,genre
1715,1771,Night Flier (1997),Horror


In [10]:
# Ez a metódus model.predict-et futtat, indexelést átképezzük (adatbázisban 1-től indul)
def predict_rating(userid, movieid):
    return trained_model.rate(userid - 1, movieid - 1)

In [12]:
# Betöltjük a felhasználói értékeléseket
user_ratings = ratings[ratings['userid'] == test_user][['userid', 'movieid', 'rating']]
# Hozzáadunk egy ajánló oszlopot és minden filmre generáljuk, hogy mennyire passzol a felhasználó ízléséhez
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiiratáshoz a predikciós és filmes táblákat egyesítjük a filmazonosító (ez közös) mentén
# Első 10-et írjuk ki
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieid', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(10)

,userid,movieid,rating,prediction,title,genre
0,4398,593,5,4.856634,"Silence of the Lambs, The (1991)",Drama|Thriller
1,4398,608,5,4.413458,Fargo (1996),Crime|Drama|Thriller
2,4398,2747,5,3.196791,"Little Shop of Horrors, The (1960)",Comedy|Horror
3,4398,1721,5,4.051601,Titanic (1997),Drama|Romance
4,4398,2248,5,4.262452,Say Anything... (1989),Comedy|Drama|Romance
5,4398,318,5,5.064855,"Shawshank Redemption, The (1994)",Drama
6,4398,1197,5,4.456388,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
7,4398,356,5,4.499308,Forrest Gump (1994),Comedy|Romance|War
8,4398,3448,5,4.005658,"Good Morning, Vietnam (1987)",Comedy|Drama|War
9,4398,2858,4,4.351923,American Beauty (1999),Comedy|Drama


In [13]:
# Szűrés azokra a filmekre, amelyeket a felhasználónk még nem látott
recommendations = ratings[ratings['movieid'].isin(user_ratings['movieid']) == False][['movieid']].drop_duplicates()
# Ajánlás a még nem látott filmekre
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiírjuk az első 10-et
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,movieid,prediction,title,genre
0,2905,4.970270,Sanjuro (1962),Action|Adventure
1,527,4.900682,Schindler's List (1993),Drama|War
2,2329,4.862729,American History X (1998),Drama
3,2762,4.795915,"Sixth Sense, The (1999)",Thriller
4,1148,4.794104,"Wrong Trousers, The (1993)",Animation|Comedy
5,50,4.752911,"Usual Suspects, The (1995)",Crime|Thriller
6,1223,4.746654,"Grand Day Out, A (1992)",Animation|Comedy
7,110,4.737195,Braveheart (1995),Action|Drama|War
8,3578,4.709396,Gladiator (2000),Action|Drama
9,745,4.691261,"Close Shave, A (1995)",Animation|Comedy|Thriller
